# English Score

Просмотр фильмов на оригинальном языке - это популярный и действенный метод прокачаться при изучении иностранных языков. Важно выбрать фильм, который подходит студенту по уровню сложности, т.ч. студент понимал 50 - 70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.

#### Задача проекта:

Разработать ML решение для автоматического определения уровня сложности англоязычных фильмов.

#### Вводные данные:
1. Файл excel с указанием фильма и разметкой уровня языковой сложности.
2. Субтитры к указанным в файле Excel.
3. Папки с субтитрами, распределенные по разным уровням сложности.

## 1. Загрузка данных

Для начала загрузим все необходимые библиотеки и имеющиеся данные, на основании которых дулее будет происходить обучение модели.

pip install pysrt

pip install pymorphy2

pip uninstall nltk

pip install -U nltk

In [1]:
import pandas as pd
from pathlib import Path
import pysrt
import codecs
from sklearn.utils import shuffle
import nltk
import numpy as np
from nltk.stem.porter import *
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool, cv
from sklearn import metrics
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yulia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- Загрузка excel с уровнями к каждому фильму

Загрузим файл `movies_labels.xlsx` и посмотрим на его содержимое.

In [2]:
df_xlsx = pd.read_excel('movies_labels.xlsx')
df_xlsx

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


Данный файл содержит в себе название фильма и уровень сложности языка к нему. Поскльку файлы с субтитрами содержат в наименовании названия фильмов, мы сможем по дынной информации соотнести каждому субтитру уровень из файла excel.

Поскольку одному фильму может быть присвоено сразу несколько уровней сложности, выберем для дальнейшего обучения самый первый указанный уровень сложности.

In [3]:
df_xlsx['Level'] = df_xlsx['Level'].str[0:2]
df_xlsx

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


- Загрузка списка .srt файлов к файлу excel

Загрузим файлы с субтитрами для фильмов из файла excel. Для этого сначала переберем полные пути к этим файлам.

In [4]:
p = Path("C:/Users/yulia/OneDrive/Документы/С рабочего стола/Проекты/Workshop_2/English_scores/Subtitles_all/Subtitles")
list_str = []

for x in p.rglob("*"):
    list_str.append(str(x))
df_srt = pd.DataFrame(list_str, columns = ['Path'])
df_srt

,Path
0,C:\Users\yulia\OneDrive\Документы\С рабочего с...
1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
3,C:\Users\yulia\OneDrive\Документы\С рабочего с...
4,C:\Users\yulia\OneDrive\Документы\С рабочего с...
...,...
111,C:\Users\yulia\OneDrive\Документы\С рабочего с...
112,C:\Users\yulia\OneDrive\Документы\С рабочего с...
113,C:\Users\yulia\OneDrive\Документы\С рабочего с...
114,C:\Users\yulia\OneDrive\Документы\С рабочего с...


Добавим колонку `Movie` и обработаем в нее названия фильмов из папки с субтитрами, чтобы далее по ней можно было подтянуть для каждого субтитра уровень из первой таблицы.

In [5]:
def add_column_movie(value):
    val = value['Path']
    val = val.replace('C:\\Users\\yulia\\OneDrive\\Документы\\С рабочего стола\\Проекты\\Workshop_2\\English_scores\\Subtitles_all\\Subtitles\\', '')     
    val = val.replace('.srt', '')     
    return val

df_srt['Movie'] = df_srt.apply(add_column_movie, axis=1)
df_srt

,Path,Movie
0,C:\Users\yulia\OneDrive\Документы\С рабочего с...,.DS_Store
1,C:\Users\yulia\OneDrive\Документы\С рабочего с...,10_Cloverfield_lane(2016)
2,C:\Users\yulia\OneDrive\Документы\С рабочего с...,10_things_I_hate_about_you(1999)
3,C:\Users\yulia\OneDrive\Документы\С рабочего с...,Aladdin(1992)
4,C:\Users\yulia\OneDrive\Документы\С рабочего с...,All_dogs_go_to_heaven(1989)
...,...,...
111,C:\Users\yulia\OneDrive\Документы\С рабочего с...,Warm_bodies(2013)
112,C:\Users\yulia\OneDrive\Документы\С рабочего с...,Westworld_scenes_of_Dr_Robert_Ford
113,C:\Users\yulia\OneDrive\Документы\С рабочего с...,We_are_the_Millers(2013)
114,C:\Users\yulia\OneDrive\Документы\С рабочего с...,While_You_Were_Sleeping(1995)


- Добавление в список excel директорий их субтитов

Получим единую таблицу, содержащую информацию об уровне фильма и местоположнии субтитра.

In [6]:
df_fin = df_xlsx.merge(df_srt, left_on='Movie', right_on='Movie')
df_fin

,id,Movie,Level,Path
0,0,10_Cloverfield_lane(2016),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
1,1,10_things_I_hate_about_you(1999),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
2,2,A_knights_tale(2001),B2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
3,3,A_star_is_born(2018),B2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
4,4,Aladdin(1992),A2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
...,...,...,...,...
105,107,Venom(2018),B2,C:\Users\yulia\OneDrive\Документы\С рабочего с...
106,108,Warm_bodies(2013),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
107,109,We_are_the_Millers(2013),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...
108,110,While_You_Were_Sleeping(1995),B1,C:\Users\yulia\OneDrive\Документы\С рабочего с...


Теперь мы можем произвести непосредственную загрузу содержимого файлов с субтритрами по их полному пути хранения. Для перебора всей таблицы сделаем отдельную функцию.

Поскольку далее нам понадобятся только колонки `Srt` с субтитрами и `Level` с уровнем английского, выберем только их. Выведем получившийся результат на экран.

In [7]:
def add_column_srt(value):
    file = str(value['Path'].replace('\\', '/'))
    text = ''    
    subs = pysrt.open(file, encoding='iso-8859-1')
    srt = ''
    for sub in subs:
        srt = srt + ' ' + sub.text
     
    return srt.replace('\n', ' ')

df_fin['Srt'] = df_fin.apply(add_column_srt, axis=1)
df_fin = df_fin[['Srt','Level']]
df_fin

,Srt,Level
0,"<font color=""#ffff80""><b>Fixed & Synced by bo...",B1
1,"Hey! I'll be right with you. So, Cameron. Her...",B1
2,Resync: Xenzai[NEF] RETAIL Should we help him...,B2
3,"- <i><font color=""#ffffff""> Synced and correc...",B2
4,"<i>Oh, I come from a land From a faraway plac...",A2
...,...,...
105,"<i>Life Foundation Control, this is LF1.</i> ...",B2
106,<i>What am I doing</i> <i>with my life?</i> <...,B1
107,"<i>Oh, my God...</i> <i>...it's full-on doubl...",B1
108,"LUCY: <i>Okay, there are two things that</i> ...",B1


- Загрузка субтитров из папок по уровням английского языка

Поскольку у нас также имеются папки по уровням английского языка, содержащие субтитры, загрузим их в отдельную таблицу с аналогичной структурой.

In [8]:
def add_column_level(value):
    val = value['Path'].split('\\')
    return val[10]

In [9]:
list_str = []
level_list = ['A2', 'B1', 'B2', 'C1']

for level in level_list:
    p = Path("C:/Users/yulia/OneDrive/Документы/С рабочего стола/Проекты/Workshop_2/English_scores/Subtitles_all"+"/"+level)
    for x in p.rglob("*"):
        list_str.append(str(x))
        
df_add = pd.DataFrame(list_str, columns = ['Path'])

df_add['Srt'] = df_add.apply(add_column_srt, axis=1)
df_add['Level'] = df_add.apply(add_column_level, axis=1)
df_add = df_add[['Srt', 'Level']]
df_add

,Srt,Level
0,( bugs chittering ) ( brakes squeak ) - ( eng...,A2
1,- ( birds chirping ) - ( bugs chittering ) Bo...,A2
2,( thunder rumbling ) Merle: That's right. You...,A2
3,( birds chirping ) - What? - Nothing. It's no...,A2
4,"- ( walkie-talkie squawks ) - Rick: Morgan, I...",A2
...,...,...
158,I lost Ava her company. I assume my deal with...,C1
159,Previously on <i>Suits...</i> It's going up o...,C1
160,"I get Ava Hessington acquitted, Darby backs m...",C1
161,Previously on <i>Suits...</i> I'm bonding wit...,C1


- Соединяем 2 таблицы

Теперь мы можем соединить две таблицы для получения большего пула данных для обучения нашей модели.

In [10]:
df_full = pd.concat([df_fin, df_add])
df_full

,Srt,Level
0,"<font color=""#ffff80""><b>Fixed & Synced by bo...",B1
1,"Hey! I'll be right with you. So, Cameron. Her...",B1
2,Resync: Xenzai[NEF] RETAIL Should we help him...,B2
3,"- <i><font color=""#ffffff""> Synced and correc...",B2
4,"<i>Oh, I come from a land From a faraway plac...",A2
...,...,...
158,I lost Ava her company. I assume my deal with...,C1
159,Previously on <i>Suits...</i> It's going up o...,C1
160,"I get Ava Hessington acquitted, Darby backs m...",C1
161,Previously on <i>Suits...</i> I'm bonding wit...,C1


Удалим все пустые значения, перемешаем данные и пересчитаем индексы.

Также первоначальные данные содержат следующие уровни английского языка: `A2`, `B1`, `B2`, `C1`. Для удобства классифицируем уровень `A2` как `A1-A2`, а `C1` как `C1-C2`. 

In [11]:
def replace_level(value):
    val = value['Level']
    val = val.replace('A2', 'A1-A2')
    val = val.replace('C1', 'C1-C2')
    return val

In [12]:
df = shuffle(df_full[df_full['Srt']!='']).reset_index(drop=True)
df['Level'] = df.apply(replace_level, axis=1)
df

,Srt,Level
0,(CROWD CHEERING) The Break-Up The Break-Up Co...,A1-A2
1,"Goddamn bugs. Oh, shit! Oh, crap. Well, Nick ...",B1
2,<b>Resync By<br/>Lututkanan@subscene</b> Do y...,B1
3,Here's something that should hurt your brain....,B1
4,"Well, I wake up in the mornin' Each and every...",B1
...,...,...
252,Asia - the largest of all the Earth's contine...,B1
253,I hear you're interviewing replacements this ...,B2
254,<i>GUARD: One con under escort. Open gate one...,C1-C2
255,(BUZZING) (BUZZING LOUDLY) VALIENTE: Show me ...,B1


## 2. Подготовка данных к обучению

- Удаление лишних символов

В рамках проведения анализов субтитров были выявлены символы и строки, не несущие в себе смысловой нагрузки и засоряющие данные для обучения модели. Это какие-либо дополнительные сведения о создателе субтитров, веб-сайты, предоставившие субтитры бесплатно, а также технические символы.

После обработки выведем наш датафрейм и посмотрим, как он изменился.

In [13]:
def replace_str(value):
    val = value['Srt']
    val = val.replace('ahmedhamdy90', ' ')
    val = val.replace('subscene.com @ahmedhamdy2121', ' ')
    val = val.replace('https://twitter.com/kaboomskull', ' ')
    val = val.replace('shokomovies.blogspot.com', ' ')
    val = val.replace('UNRATED.720p.BluRay.x264-REFiNED.English Upload to subscene.com by ViSTAâ\x84¢-HDVN', ' ')
    val = val.replace('www.YIFY-TORRENTS.com', ' ')
    val = val.replace('www.ncicap.org', ' ')
    val = val.replace('truecostmovie.com', ' ')
    val = val.replace('Created and Encoded by --  Bokutox -- of  www.YIFY-TORRENTS.com. The Best 720p/1080p/3d movies with the lowest file size on the internet.', ' ')
    val = val.replace('www.facemash.com', ' ')
    val = val.replace('www.OpenSubtitles.org', ' ')
    val = val.replace('www.titlovi.com', ' ')
    val = val.replace('www.AllSubs.org', ' ')
    val = val.replace('www.flixify.app', ' ')
    val = val.replace('truecostmovie.com', ' ')
    val = val.replace('teenybikini.com', ' ')

    val = val.lower()

    val = val.replace('font', ' ')
    val = val.replace('color', ' ')
    val = val.replace('ffff', ' ')
    val = val.replace('<b>', ' ')
    val = val.replace('</b>', ' ')
    val = val.replace('<i>', ' ')
    val = val.replace('</i>', ' ')
    val = val.replace('font color', ' ')
    val = val.replace('</font>', ' ')
    val = val.replace('â\x99ª', ' ')
    val = val.replace('d900d9', ' ')
    val = val.replace('âª', ' ')
    
    return val

df['Srt'] = df.apply(replace_str, axis=1)
df

,Srt,Level
0,(crowd cheering) the break-up the break-up co...,A1-A2
1,"goddamn bugs. oh, shit! oh, crap. well, nick ...",B1
2,resync by<br/>lututkanan@subscene do you ha...,B1
3,here's something that should hurt your brain....,B1
4,"well, i wake up in the mornin' each and every...",B1
...,...,...
252,asia - the largest of all the earth's contine...,B1
253,i hear you're interviewing replacements this ...,B2
254,guard: one con under escort. open gate one. ...,C1-C2
255,(buzzing) (buzzing loudly) valiente: show me ...,B1


- Лемматизация

Лемматизация – это алгоритмический процесс нахождения леммы слова в зависимости от его значения. Лемматизация обычно относится к морфологическому анализу слов, целью которого является удаление флективных окончаний. Это помогает в возвращении базовой или словарной формы слова, которое известно как лемма. Метод лемматизации NLTK основан на встроенной морф-функции WorldNet. Предварительная обработка текста включает в себя как основы, так и лемматизации.

Проведем лемматизацию для наших субтитров, а также избавимся от лишних символов.

In [14]:
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_en = stopwords.words("english")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_en:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

df['Srt'] = df['Srt'].apply(lemmatize)
df

,Srt,Level
0,"[crowd, cheering, break, break, come, come, co...",A1-A2
1,"[goddamn, bugs, oh, shit, oh, crap, well, nick...",B1
2,"[resync, br, lututkanan, subscene, idea, br, a...",B1
3,"[something, hurt, brain, empty, half, beds, ho...",B1
4,"[well, wake, mornin, every, day, sit, table, h...",B1
...,...,...
252,"[asia, largest, earth, continents, stretches, ...",B1
253,"[hear, interviewing, replacements, morning, go...",B2
254,"[guard, one, con, escort, open, gate, one, gat...",C1-C2
255,"[buzzing, buzzing, loudly, valiente, show, got...",B1


In [15]:
def lem_back(plurals):
    singles = [plural for plural in plurals]
    return (' '.join(singles))

df['Srt'] = df['Srt'].apply(lem_back)
df

,Srt,Level
0,crowd cheering break break come come come righ...,A1-A2
1,goddamn bugs oh shit oh crap well nick dick su...,B1
2,resync br lututkanan subscene idea br arguing ...,B1
3,something hurt brain empty half beds hospitals...,B1
4,well wake mornin every day sit table hear dadd...,B1
...,...,...
252,asia largest earth continents stretches equato...,B1
253,hear interviewing replacements morning good gr...,B2
254,guard one con escort open gate one gate opens ...,C1-C2
255,buzzing buzzing loudly valiente show got bones...,B1


- Оценка релевантности слов с помощью частотно-обратной частоты термина в документе

Теперь переведем получившиеся данные в понятный модели язык - оценим релевантность слов с помощью частотно-обратной частоты термина в документе, используя для этого `TfidfTransformer`.

In [16]:
docs = np.array(df['Srt'])
pipe = Pipeline([('count', CountVectorizer()),
                ('tfid', TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True))]).fit(docs)

In [17]:
tfidf = pd.DataFrame(pipe.transform(docs).toarray())
df_srt = pd.DataFrame(tfidf)
df_srt['Level'] = df['Level']
df_srt

,0,1,2,3,4,5,6,7,8,9,...,33418,33419,33420,33421,33422,33423,33424,33425,33426,Level
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.007707,0.0,0.0,0.0,0.0,0.0,0.0,A1-A2
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,B1
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,B1
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,B1
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,B1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,B1
253,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,B2
254,0.007783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,C1-C2
255,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,B1


## 3. Обучение модели
Признаком для обучения нашей модели будут преобразованные субтитры, целевой признак - уровни английского языка из колонки `Level`. Разобьем данные на обучающие и тестовые.

In [18]:
X = df_srt.drop(['Level'], axis=1)
y = df_srt['Level']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12345)

В рамках данного проекта был выбран инструмент `CatBoostClassifier`, который поможет нам решить задачу мультиклассификации.
Для этого поместим признаки в `Pool` для обучения нашей модели.

In [19]:
learn_pool = Pool(
    X_train, 
    y_train
)

test_pool = Pool(
    X_test, 
    y_test
)

- Кросс-валидация и подбор гипер-параметра

Используем кросс-валидацию от CatBoost с `3` бакетами и подберем гиперпараметр `learning_rate` с наиболее высоким показателем среднего `F1` по результатам кросс-валидации.

In [20]:
lr_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for i in lr_list:
    params = {
    'learning_rate': i,
    'iterations': 1000,
    'eval_metric': 'TotalF1',
    'early_stopping_rounds': 200,
    'use_best_model': True,
    'verbose': 100,
    'loss_function': 'MultiClass'
    }
    
    print()
    print()
    print('learning_rate:', i)
    print()
    scores = cv(learn_pool,
                params,
                fold_count=3,
                )
    
    print(scores[scores['test-TotalF1-mean'] == scores['test-TotalF1-mean'].max()])



learning_rate: 0.1

Training on fold [0/3]
0:	learn: 0.6084210	test: 0.3181818	best: 0.3181818 (0)	total: 1.32s	remaining: 22m 3s
100:	learn: 0.9920265	test: 0.4964034	best: 0.4970896 (91)	total: 1m 40s	remaining: 14m 53s
200:	learn: 0.9920265	test: 0.5550661	best: 0.5559157 (161)	total: 3m 15s	remaining: 12m 56s
300:	learn: 0.9920978	test: 0.5550661	best: 0.5559157 (161)	total: 4m 55s	remaining: 11m 26s

bestTest = 0.5559157352
bestIteration = 161

Training on fold [1/3]
0:	learn: 0.5844597	test: 0.4078664	best: 0.4078664 (0)	total: 1.06s	remaining: 17m 38s
100:	learn: 1.0000000	test: 0.5207888	best: 0.5362592 (95)	total: 1m 44s	remaining: 15m 27s
200:	learn: 1.0000000	test: 0.5596967	best: 0.5732590 (142)	total: 3m 23s	remaining: 13m 30s
300:	learn: 1.0000000	test: 0.5822030	best: 0.6036005 (224)	total: 5m 1s	remaining: 11m 39s
400:	learn: 1.0000000	test: 0.6036005	best: 0.6036005 (224)	total: 6m 42s	remaining: 10m 1s

bestTest = 0.6036005435
bestIteration = 224

Training on fold [

0:	learn: 0.6084210	test: 0.3181818	best: 0.3181818 (0)	total: 979ms	remaining: 16m 18s
100:	learn: 0.9920978	test: 0.5413226	best: 0.5413226 (75)	total: 1m 40s	remaining: 14m 54s
200:	learn: 0.9920978	test: 0.5770202	best: 0.5770202 (153)	total: 3m 14s	remaining: 12m 54s
300:	learn: 0.9920265	test: 0.5573517	best: 0.5770202 (153)	total: 4m 53s	remaining: 11m 21s
400:	learn: 0.9920978	test: 0.5573517	best: 0.5848253 (346)	total: 6m 28s	remaining: 9m 40s
500:	learn: 0.9920265	test: 0.5855699	best: 0.5855699 (488)	total: 8m 4s	remaining: 8m 2s
600:	learn: 0.9920265	test: 0.5848253	best: 0.5855699 (488)	total: 9m 43s	remaining: 6m 27s

bestTest = 0.5855698899
bestIteration = 488

Training on fold [1/3]
0:	learn: 0.5844597	test: 0.4078664	best: 0.4078664 (0)	total: 993ms	remaining: 16m 32s
100:	learn: 1.0000000	test: 0.5816801	best: 0.5816801 (100)	total: 1m 44s	remaining: 15m 32s
200:	learn: 1.0000000	test: 0.6317498	best: 0.6317498 (183)	total: 3m 20s	remaining: 13m 18s
300:	learn: 1.000

100:	learn: 0.9920978	test: 0.4807296	best: 0.5014436 (98)	total: 1m 41s	remaining: 15m 1s
200:	learn: 0.9920265	test: 0.5035354	best: 0.5058804 (132)	total: 3m 16s	remaining: 13m 1s
300:	learn: 0.9920265	test: 0.5058804	best: 0.5058804 (132)	total: 4m 52s	remaining: 11m 20s

bestTest = 0.5058803635
bestIteration = 132

Training on fold [1/3]
0:	learn: 0.5844597	test: 0.4078664	best: 0.4078664 (0)	total: 982ms	remaining: 16m 20s
100:	learn: 1.0000000	test: 0.5283283	best: 0.5522978 (61)	total: 1m 41s	remaining: 15m 6s
200:	learn: 1.0000000	test: 0.5964744	best: 0.6303158 (189)	total: 3m 16s	remaining: 13m 2s
300:	learn: 1.0000000	test: 0.5964744	best: 0.6303158 (189)	total: 4m 56s	remaining: 11m 27s

bestTest = 0.6303158471
bestIteration = 189

Training on fold [2/3]
0:	learn: 0.5361439	test: 0.3880136	best: 0.3880136 (0)	total: 1.07s	remaining: 17m 49s
100:	learn: 1.0000000	test: 0.5483084	best: 0.5483084 (98)	total: 1m 40s	remaining: 14m 53s
200:	learn: 1.0000000	test: 0.5263930	best

200:	learn: 1.0000000	test: 0.5156469	best: 0.5408955 (157)	total: 3m 26s	remaining: 13m 39s
300:	learn: 1.0000000	test: 0.5408955	best: 0.5408955 (157)	total: 5m 7s	remaining: 11m 52s

bestTest = 0.5408955458
bestIteration = 157

     iterations  test-TotalF1-mean  test-TotalF1-std  train-TotalF1-mean  \
191         191           0.556324          0.034818            0.997342   
192         192           0.556324          0.034818            0.997342   
193         193           0.556324          0.034818            0.997342   

     train-TotalF1-std  test-MultiClass-mean  test-MultiClass-std  \
191           0.004603              1.211266             0.219364   
192           0.004603              1.210718             0.218967   
193           0.004603              1.211919             0.219745   

     train-MultiClass-mean  train-MultiClass-std  
191               0.010078              0.008647  
192               0.010042              0.008643  
193               0.010018        

По результатам кросс-валидации лучше всех показал себя гипер-параметр `learning_rate` со значением `0.5` - F1-мера составила `0.5789`.
Зададим основные параметры модели, обучим ее и посмотрим на результаты по категориям.

In [31]:
params = {
    'iterations': 1000,
    'learning_rate': 0.5,
    'eval_metric': 'TotalF1',
    'early_stopping_rounds': 200,
    'use_best_model': True,
    'verbose': 100,
    'loss_function': 'MultiClass',
    'score_function': 'L2'
}

In [33]:
model = CatBoostClassifier(**params)
model.fit(learn_pool, eval_set=test_pool)
print()
predictions = model.predict(test_pool)
print()
print(metrics.classification_report(y_test, predictions, digits=3))

0:	learn: 0.7081878	test: 0.5910731	best: 0.5910731 (0)	total: 13.6s	remaining: 3h 46m 39s
100:	learn: 0.9948063	test: 0.6706111	best: 0.6706111 (75)	total: 2m 11s	remaining: 19m 33s
200:	learn: 0.9948063	test: 0.6827628	best: 0.6827628 (138)	total: 4m 3s	remaining: 16m 7s
300:	learn: 0.9948063	test: 0.6827628	best: 0.6827628 (138)	total: 6m 6s	remaining: 14m 12s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.6827627757
bestIteration = 138

Shrink model to first 139 iterations.


              precision    recall  f1-score   support

       A1-A2      0.600     0.375     0.462         8
          B1      0.778     0.438     0.560        16
          B2      0.700     0.921     0.795        38
       C1-C2      1.000     0.333     0.500         3

    accuracy                          0.708        65
   macro avg      0.769     0.517     0.579        65
weighted avg      0.721     0.708     0.683        65



Максимальный `F1`составил `0.68`, при этом мы видим, что, в целом, по категории `A1-A2` показатель `F1` не дотягивает даже до значения `0.5`. Это значит, что по данной категонии наша модель предсказывает хуже, чем предсказала бы брошенная монетка.

## 4. Предсказание уровня английского языка для загружаемых субтитров

Пожалуйста, укажите путь к вашему файлу с субтитрами (.srt):

In [50]:
path = r'C:\Users\yulia\OneDrive\Документы\С рабочего стола\Проекты\Workshop_2\English_scores\Test\Toy_Story_1995_2160p_UHD_BluRay_REMUX_HDR10_HEVC_Atmos_7_1_OMEGA.srt'

In [51]:
# Загрузка .srt
# Создать df с колокой Path, вставить туда путь к файлу
df_check = pd.DataFrame({'Path': [path]})
df_check['Srt'] = df_check.apply(add_column_srt, axis=1)

df_check['Srt'] = df_check.apply(replace_str, axis=1)

# Лемматизация
df_check['Srt'] = df_check['Srt'].apply(lemmatize)
df_check['Srt'] = df_check['Srt'].apply(lem_back)

docs_ch = np.array(df_check['Srt'])

df_ch = pd.DataFrame(pipe.transform(docs_ch).toarray())

pool = Pool(
    df_ch
)

# Предсказание
prediction = model.predict(pool)
print(prediction)

[['A1-A2']]


# Вывод
В рамках данного проекта:
- Получены, загружены и обработаны вводные данные.
- Проведена очистка текстового содержания, лемматизация и избавление от стоп-слов.
- Оценена релевантность слов с помощью частотно-обратной частоты термина в документе с помощью tfidf.
- Проведена кросс-валидация с оценкой средней `F1`.
- Реализовано ML решение, позволяющее предсказывать уровень английского языка по загружаемым субтитрам.
- Для проверки загружаемого субтитра добавлена возможность использовать tfidf на базе данных обучения с поощью `pipeline`.

В рамках анализа результатов различных подходов результат `F1`в размере `0.68` остается наиболее высоким, при этом по категории `A1-A2` показатель `F1` не дотягивает даже до значения `0.5`. Поэтому, для улучшения качества модели, как область для развития проекта, мы видим возможность рассмотреть другие подходы для лемматизации, оценки релевантности слов, а также более глубокого анализа при чистке субтитров.